In [602]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [582]:
#IMPORT LIBRARIES

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms
from sklearn.metrics import roc_auc_score
seed = 42
np.random.seed(42)

In [583]:
train_data = pd.read_csv('/content/drive/MyDrive/camerai/Train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/camerai/Test.csv')

In [584]:
train_data.head()

,img_IDs,target
0,ID_04UTZ83A24,0
1,ID_0URH2U2RON,1
2,ID_0ZEPOZGGQ1,0
3,ID_1595QTX5OA,1
4,ID_1RAE4SS5S6,1


In [585]:
images = '/content/drive/MyDrive/camerai/images/CAMAIRAI BOX READY'

In [586]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

In [587]:
# Function to load images from the specified directory
def load_image(img_id):
    img_path = os.path.join('/content/drive/MyDrive/camerai/images/CAMAIRAI BOX READY', f'{img_id}.jpg')
    return transform(Image.open(img_path))

In [588]:
# Load and preprocess images
train_images = [load_image(img_id) for img_id in train_data['img_IDs']]
test_images = [load_image(img_id) for img_id in test_data['img_IDs']]

In [589]:
# Convert target labels to numpy array
train_labels = np.array(train_data['target'])

In [590]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=seed)

In [591]:
# Create DataLoader for training and validation
train_dataset = TensorDataset(torch.stack(X_train), torch.Tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(torch.stack(X_val), torch.Tensor(y_val))
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [592]:
# Define the CNN model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Sigmoid()
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [593]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [594]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
epochs = 78

In [595]:
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()  # Squeeze to match the label shape
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [596]:
# Model evaluation
model.eval()
val_preds = []
val_labels = []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs).squeeze()
        val_preds.extend(outputs.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

In [597]:
# Calculate AUC for validation set
val_auc = roc_auc_score(val_labels, val_preds)
print(f'Validation AUC: {val_auc}')

Validation AUC: 0.9166666666666667


In [598]:
# Make predictions on the test dataset
test_dataset = TensorDataset(torch.stack(test_images))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
test_preds = []

In [599]:
model.eval()
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs[0].to(device)
        outputs = model(inputs).squeeze()
        test_preds.extend(outputs.cpu().numpy())

In [603]:
# Create a submission file
submission_df = pd.DataFrame({'ID': test_data['img_IDs'], 'target': test_preds})
submission_df.to_csv('/content/drive/MyDrive/camerai/cameroon_2_91.667_submission.csv', index=False)